In [16]:
import os
import sys
import joblib
import numpy as np
import pandas as pd
from tqdm import tqdm
import warnings
warnings.filterwarnings(action='ignore')

In [35]:
data = joblib.load(os.path.join('..', '..', '0.Data', '05_분석데이터', 'Thd_FE_202009062207.pkl'))

In [36]:
locals().update(data)

In [13]:
drop_columns = ['결제방법', '상품군_가격대', '전체_가격대', '상품군', '방송시간(시간)', '성별']

In [14]:
X.drop(drop_columns, axis = 1)

KeyError: "['결제방법' '상품군_가격대' '전체_가격대' '상품군' '방송시간(시간)' '성별'] not found in axis"

In [7]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [6]:
# vif 
vif = pd.DataFrame()
vif['VIF Factor'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif['features'] = X.columns

NameError: name 'variance_inflation_factor' is not defined